In [16]:
import ast
from pathlib import Path

import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

In [18]:
root = Path('/media/t9s9/SSD_ubuntu/datasets/WavText5k/')
df = pd.read_csv(root / 'WavText5K.csv')
df['filename'] = df.fname.apply(lambda f: str(Path('train') / Path(f).with_suffix('.flac')))

exists = df.filename.apply(lambda p: (root / 'data' / p).exists())
df = df[exists]

df = df.drop(columns=['download_link', 'view_link', 'fname'])
df = df.dropna(subset=['title', 'description', 'tags'], how='all')
df.tags = df.tags.replace(pd.NA, '[]')

bad = "<li><i class=\"icofont-sound-wave icofont-1x\"></i> <a href=\"tags-alcohol-bashed.html\">alcohol bashed</a></li>,"
bad2 = "<li><i class=\"icofont-sound-wave icofont-1x\"></i> <a href=\"tags-bow-and-arrow.html\">bow and arrow</a></li>,"
df.tags = df.tags.str.replace(bad, '')
df.tags = df.tags.str.replace(bad2, '')
df.tags = df.tags.apply(ast.literal_eval)
df

/tmp/ipykernel_66908/406120864.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df.tags = df.tags.str.replace(bad, '')
/tmp/ipykernel_66908/406120864.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df.tags = df.tags.str.replace(bad2, '')


,title,description,tags,filename
0,Forest,"Birds singing, drive away, fly.",[],train/0100.flac
1,Rain and Storm 2,Heavy rain and thunderstorms.,[],train/0740.flac
2,Wilhelm scream,NaN,[],train/0477.flac
3,Wind,NaN,[],train/0595.flac
4,Waves and Sandwich tern,Sea waves at medium volume and a few cries of ...,[],train/0267.flac
...,...,...,...,...
4520,Bat Sonar,Bat sonar chirping clicks,"[bat, call, inside, cave, sonar, bats, rodent,...",train/bat sonar_2110.flac
4521,Bat Call Inside Cave,Bat call inside cave,"[bat, call, inside, cave, sonar, bats, rodent]",train/bat call inside cave_2111.flac
4522,Baby Monitor,Cute Kid talking over baby monitor with static,"[static noise, static, kid, monitor, baby, tal...",train/baby monitor_2112.flac
4523,Baby Girl Speaking,Baby Girl Saying Mama,"[Baby, Girl, Speaking, child, kid, young, mom,...",train/baby girl speaking_2113.flac


In [19]:
def create_caption(row):
    title, desc, tags = row.title, row.description, row.tags
    has_title, has_desc = not pd.isna(title), not pd.isna(desc)
    has_tags = tags != []

    if has_title:
        title = ''.join(filter(lambda x: not x.isdigit(), title)).strip()

    if not has_tags:
        if has_title:
            tags = [token.text for token in nlp(title) if not token.is_stop and not token.is_punct]
        elif has_desc:
            tags = [token.text for token in nlp(desc) if not token.is_stop and not token.is_punct]
        else:
            print("Warning: No tags can be formed")

    if has_desc:
        caption = f"{desc.strip().rstrip('.')}."
    elif has_title:
        caption = f"{title}."
    elif has_tags:
        print("Warning: Caption made from tags")
        caption = f"{' '.join(row.tags)}."
    else:
        print("Warning: No caption formed")
        caption = ''


    return [caption], tags

df[['captions', 'tags']] = df.apply(create_caption, axis=1, result_type='expand')
df

,title,description,tags,filename,captions
0,Forest,"Birds singing, drive away, fly.",[Forest],train/0100.flac,"[Birds singing, drive away, fly.]"
1,Rain and Storm 2,Heavy rain and thunderstorms.,"[Rain, Storm]",train/0740.flac,[Heavy rain and thunderstorms.]
2,Wilhelm scream,NaN,"[Wilhelm, scream]",train/0477.flac,[Wilhelm scream.]
3,Wind,NaN,[Wind],train/0595.flac,[Wind.]
4,Waves and Sandwich tern,Sea waves at medium volume and a few cries of ...,"[Waves, Sandwich, tern]",train/0267.flac,[Sea waves at medium volume and a few cries of...
...,...,...,...,...,...
4520,Bat Sonar,Bat sonar chirping clicks,"[bat, call, inside, cave, sonar, bats, rodent,...",train/bat sonar_2110.flac,[Bat sonar chirping clicks.]
4521,Bat Call Inside Cave,Bat call inside cave,"[bat, call, inside, cave, sonar, bats, rodent]",train/bat call inside cave_2111.flac,[Bat call inside cave.]
4522,Baby Monitor,Cute Kid talking over baby monitor with static,"[static noise, static, kid, monitor, baby, tal...",train/baby monitor_2112.flac,[Cute Kid talking over baby monitor with static.]
4523,Baby Girl Speaking,Baby Girl Saying Mama,"[Baby, Girl, Speaking, child, kid, young, mom,...",train/baby girl speaking_2113.flac,[Baby Girl Saying Mama.]


In [21]:
df = df.drop(columns=['description', 'title'])
df = df.drop(index=df[df.tags.apply(len) == 0].index)

KeyError: "['description', 'title'] not found in axis"

In [23]:
df = pd.read_parquet('/media/t9s9/SSD_ubuntu/datasets/WavText5k/annot.parquet')
df

,tags,filename,captions
0,[Forest],train/0100.flac,"[Birds singing, drive away, fly.]"
1,"[Rain, Storm]",train/0740.flac,[Heavy rain and thunderstorms.]
2,"[Wilhelm, scream]",train/0477.flac,[Wilhelm scream.]
3,[Wind],train/0595.flac,[Wind.]
4,"[Waves, Sandwich, tern]",train/0267.flac,[Sea waves at medium volume and a few cries of...
...,...,...,...
4514,"[bat, call, inside, cave, sonar, bats, rodent,...",train/bat sonar_2110.flac,[Bat sonar chirping clicks.]
4515,"[bat, call, inside, cave, sonar, bats, rodent]",train/bat call inside cave_2111.flac,[Bat call inside cave.]
4516,"[static noise, static, kid, monitor, baby, tal...",train/baby monitor_2112.flac,[Cute Kid talking over baby monitor with static.]
4517,"[Baby, Girl, Speaking, child, kid, young, mom,...",train/baby girl speaking_2113.flac,[Baby Girl Saying Mama.]


In [22]:
df.to_parquet('/media/t9s9/SSD_ubuntu/datasets/WavText5k/annot.parquet', index=False)
df

,tags,filename,captions
0,[Forest],train/0100.flac,"[Birds singing, drive away, fly.]"
1,"[Rain, Storm]",train/0740.flac,[Heavy rain and thunderstorms.]
2,"[Wilhelm, scream]",train/0477.flac,[Wilhelm scream.]
3,[Wind],train/0595.flac,[Wind.]
4,"[Waves, Sandwich, tern]",train/0267.flac,[Sea waves at medium volume and a few cries of...
...,...,...,...
4520,"[bat, call, inside, cave, sonar, bats, rodent,...",train/bat sonar_2110.flac,[Bat sonar chirping clicks.]
4521,"[bat, call, inside, cave, sonar, bats, rodent]",train/bat call inside cave_2111.flac,[Bat call inside cave.]
4522,"[static noise, static, kid, monitor, baby, tal...",train/baby monitor_2112.flac,[Cute Kid talking over baby monitor with static.]
4523,"[Baby, Girl, Speaking, child, kid, young, mom,...",train/baby girl speaking_2113.flac,[Baby Girl Saying Mama.]
